<a href="https://colab.research.google.com/github/horsinnaround/Data-Science-Tools/blob/main/Tools_and_other_Stuff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing

In [ ]:
from io import StringIO
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
import numpy as np
from sklearn.model_selection import cross_val_score
import zipfile
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from numpy import array
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB 
!pip install catboost
from xgboost import XGBClassifier
from catboost import CatBoostClassifier 
import pandas as pd
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import seaborn as sns
from time import time


# All the way with texts

## Text to int

for changing dtype of data frame columns to serial or factorize them or make strings ti ints

In [ ]:
# df_ = put dataframe here
str_to_fac=[]
for i in range(len(df.dtypes)):
    if df.dtypes[i] == 'object':
        to_fac = df[df.columns[i]].factorize()
        str_to_fac.append(to_fac)
        df_[f'{df.columns[i]}'] = to_fac[0]

## Text to vector

In [ ]:
# text to vector
# from sklearn.feature_extraction.text import TfidfVectorizer
# text is the column that you have the sentense
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df.text).toarray()
labels = df.category_id
features.shape

# Regular ml

## classification with different data sample number

In [ ]:
%%time

# Learning curve:

scores_list = []

clf = CatBoostClassifier()
# clf = SVC()

def loss_func(Y_true, Y_prediction): # definig a loss function
    return (1 - accuracy_score(Y_true, Y_prediction))

ns_list = [1000,5000,9000,13000,19000,25000] # number of samples that we want to keep
n_avg = 1  # Number of times to average over

labels = list(df['category_id'])
for ns in ns_list:   # For different number of samples, we try training our estimator
    print(f"for {ns} number of data")        

    e_in = 0
    e_out = 0   # For each, we record the in and out score. 
    for i in range(n_avg):
        X_train, X_test, Y_train, Y_test = train_test_split(features[0:ns],labels[0:ns], random_state = 0)

        clf.fit( X_train , Y_train )

        e_in  += loss_func( Y_train, clf.predict(X_train ) ) # in-sample error
        e_out += loss_func( Y_test , clf.predict(X_test )  ) # out-sample error

    scores_list += [[e_in/n_avg, e_out/n_avg]]

    
scores_list = array(scores_list)  

xlabel = 'Training samples'
ylabel = 'loss'
plt_title = 'Learning curve for classification data \n using a CatBoost Classifier'
fig = plt.figure(figsize=(8,6),dpi=300)
ax2 = fig.add_subplot(111, xlabel=xlabel, ylabel=ylabel, title=plt_title)
ax2.plot(ns_list, scores_list[:,0], '-o',label = 'Training Score')
ax2.plot(ns_list, scores_list[:,1], '-o',label = 'Validation Score')
ax2.legend()
plt.savefig('learning curve.png')
plt.show()

# Data

## remove data with very low sample

In [ ]:
features = df_p.drop('labels',axis=1)
labels = df_p['labels']
labels_t = Counter(labels)
print('Original dataset shape %s' % Counter(labels))
drop_dead = []
for lab in labels_t.keys():
    if len(df[df['labels']==lab])<300:
        drop_dead.append(lab)
        df.drop(index=df[df['labels']==lab].index,inplace=True)
print('Final dataset shape %s' % Counter(labels))

## change labels in dataframe

In [ ]:
index = df[df.labels!='normal'].index
df.loc[index] = 'nep'

## zip unzipping

In [ ]:
# Unzipping
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/A/hamed thesis/kdd_test.csv.zip', "r")
zip_ref.extractall('/content/drive/MyDrive/A/hamed thesis/')
zip_ref.close()

## PCA

In [ ]:
# from sklearn.decomposition import PCA
pca = PCA(n_components=100, copy=True, whiten=False, svd_solver='auto', tol=0.0, iterated_power='auto',random_state=None)
features = pca.fit_transform(features)
features.shape

## barplot

In [ ]:
# Count Plot (a.k.a. Bar Plot)
plt.figure(figsize=(9,6),dpi=200)
sns.countplot(y='labels', data=df_)
# plt.xticks(rotation=-90)

## cor plot

In [ ]:
corr = df_.corr();
plt.figure(figsize=(9,9),dpi=100);
# sns.heatmap(corr,linewidths=.5);
sns.heatmap(corr);

plt.xticks(rotation=-90);
plt.yticks(rotation=0);

## Smothe

In [ ]:
from imblearn.over_sampling import SVMSMOTE 
print('Original dataset shape %s' % Counter(y))
sm = SVMSMOTE(sampling_strategy='auto', 
              random_state=1, 
              k_neighbors=5, 
              n_jobs=None, 
              m_neighbors=10, 
              svm_estimator=None,
              out_step=0.5)
X_t, y_t = sm.fit_resample(X_t, y_t)
print('Resampled dataset shape %s' % Counter(y_t))

## Test classification

In [ ]:
from collections import Counter
from sklearn.datasets import make_classification

X, y = make_classification(n_classes=2, class_sep=2,
weights=[0.1, 0.9], n_informative=3, n_redundant=1, flip_y=0,
n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)
print('Original dataset shape %s' % Counter(y))
print('Resampled dataset shape %s' % Counter(y_t))